## ひろしまQuest2020#stayhome【球種予測部門】
### ストレートか変化球かを予測するサブモデル

#### all_merge_19
- コースの種類のみ

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import lightgbm as lgb
import gc
%matplotlib inline
pd.set_option('display.max_Columns', 100)

### 管理番号

In [2]:
model_No = 19

In [3]:
ALL_MERGE = 'intermediate/all_merge_{}.f'.format(model_No)
OUT_SUBMODEL = 'intermediate/binary_predict_{}.f'.format(model_No)

In [4]:
all_pitch = pd.read_feather(ALL_MERGE)
all_pitch.shape

(778767, 190)

### 球種

In [5]:
all_pitch['binary_straight'] = 0
all_pitch.loc[all_pitch['ball'] == 0, 'binary_straight'] = 1
all_pitch['binary_curve'] = 0
all_pitch.loc[all_pitch['ball'] == 1, 'binary_curve'] = 1
all_pitch['binary_slider'] = 0
all_pitch.loc[all_pitch['ball'] == 2, 'binary_slider'] = 1
all_pitch['binary_shoot'] = 0
all_pitch.loc[all_pitch['ball'] == 3, 'binary_shoot'] = 1
all_pitch['binary_fork'] = 0
all_pitch.loc[all_pitch['ball'] == 4, 'binary_fork'] = 1
all_pitch['binary_changeup'] = 0
all_pitch.loc[all_pitch['ball'] == 5, 'binary_changeup'] = 1
all_pitch['binary_sinker'] = 0
all_pitch.loc[all_pitch['ball'] == 6, 'binary_sinker'] = 1
all_pitch['binary_cutball'] = 0
all_pitch.loc[all_pitch['ball'] == 7, 'binary_cutball'] = 1

In [6]:
print(all_pitch.shape)
column_cnt = len(all_pitch.columns)
all_pitch.head()

(778767, 198)


,No,ball,course,pitch_cnt_in_game,inning,bat_cnt_in_inning,pitch_cnt_in_bat,pitch_order,player_cnt_in_game,pitcher_cnt_in_game,pitcher_cnt_in_inning,bat_order,bat_cnt_in_game,home_point,away_point,out_cnt,ball_cnt,strike_cnt,pitch_LR,bat_LR,bc_curve,bc_slider,bc_shoot,bc_fork,bc_changeup,bc_sinker,bc_cutball,bc_high_str,bc_high_ball,bc_mid_str,bc_low_str,bc_low_ball,bc_left_str,bc_left_ball,bc_center_str,bc_right_str,bc_right_ball,first,second,third,base_cnt,top_bot,role,pos_pit,date_from_opening,elapsed_min,min_diff,total_inning,elapsed_from_inning,ballnum_from_inning,...,draft_order,salary,foreigner,company,univ,highsch,age,play_year,salary_rank,rank_year,rank_x_year,salary_year,salary_x_year,bmi,salary_dif_p-b,play_year_dif_p-b,age_dif_p-b,salary_year_dif_p-b,salary_x_year_dif_p-b,rank_year_dif_p-b,rank_x_year_dif_p-b,bmi_dif_p-b,salary_dif_p-c,play_year_dif_p-c,age_dif_p-c,salary_year_dif_p-c,salary_x_year_dif_p-c,rank_year_dif_p-c,rank_x_year_dif_p-c,bmi_dif_p-c,salary_dif_b-c,play_year_dif_b-c,age_dif_b-c,salary_year_dif_b-c,salary_x_year_dif_b-c,rank_year_dif_b-c,rank_x_year_dif_b-c,bmi_dif_b-c,pit_bat_L_L,pit_bat_L_R,pit_bat_R_L,pit_bat_R_R,binary_straight,binary_curve,binary_slider,binary_shoot,binary_fork,binary_changeup,binary_sinker,binary_cutball
0,0,0.0,0.0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0.0,1.0,0.237468,0.325101,0.209547,0.145227,0.092978,0.027829,0.113620,0.135883,0.221489,0.204959,0.174254,0.263415,0.242045,0.294634,0.157681,0.115370,0.190270,0,0,0,0,0,1,0,0,0.000000,NaN,0,0.000000,1,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,1,0,0,0,0,0,0,0
1,1,0.0,8.0,2,1,1,2,1,1,2,2,1,1,0,0,0,0,1,0.0,1.0,0.210923,0.247198,0.177502,0.320359,0.147952,0.039943,0.116772,0.106283,0.230235,0.157397,0.138195,0.367890,0.184621,0.355720,0.111331,0.105923,0.242405,0,0,0,0,0,1,0,0,0.200000,0.200000,0,0.200000,2,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,1,0,0,0,0,0,0,0
2,2,0.0,5.0,3,1,1,3,1,1,3,3,1,1,0,0,0,0,2,0.0,1.0,0.138758,0.130040,0.092263,0.419906,0.117327,0.038140,0.046858,0.076007,0.284249,0.102015,0.095055,0.442674,0.118315,0.488095,0.076190,0.078571,0.238828,0,0,0,0,0,1,0,0,0.383333,0.183333,0,0.383333,3,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,9000,-2.0,-3,2125.0,24000.0,0.604167,13.0,2.141358,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,10000,4.0,4,625.0,100000.0,-5.562500,-74.0,-2.306725,0,0,1,0,1,0,0,0,0,0,0,0
3,3,0.0,12.0,4,1,2,1,1,2,4,4,2,1,0,0,1,0,0,0.0,0.0,0.213866,0.474931,0.139964,0.088519,0.038543,0.007023,0.138168,0.151308,0.196937,0.206965,0.157643,0.287147,0.105057,0.153718,0.164756,0.246104,0.330367,0,0,0,0,0,1,0,0,1.016667,0.633333,0,1.016667,4,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1,1,0,0,0,0,0,0,0
4,4,0.0,8.0,5,1,2,2,1,2,5,5,2,1,0,0,1,0,1,0.0,0.0,0.215241,0.420099,0.145723,0.196762,0.051566,0.009680,0.118444,0.113468,0.206297,0.162901,0.127988,0.389347,0.084346,0.162167,0.120564,0.199445,0.433477,0,0,0,0,0,1,0,0,1.200000,0.183333,0,1.200000,5,...,1.0,5000,0,1,1,0,28,4.0,23.5,5.875,94.0,1250.0,20000.0,26.196187,16960,-16.0,-11,3680.0,-10880.0,0.325758,-253.0,3.191314,19000,2.0,1,2750.0,124000.0,-4.958333,-61.0,-0.165367,2040,18.0,12,-930.0,134880.0,-5.284091,192.0,-3.356681,0,0,0,1,1,0,0,0,0,0,0,0


### train
- 行数: 257117

In [7]:
train = all_pitch.dropna(subset=['course'])
train.shape

(257117, 198)

In [8]:
train_d = train.drop([
    'No', 
    'course', 
    'ball',
    'binary_straight','binary_curve','binary_slider','binary_shoot',
    'binary_fork','binary_changeup','binary_sinker','binary_cutball'
], axis=1)

### 全体で予測する

In [9]:
all_d = all_pitch.drop([
    'No', 
    'course', 
    'ball',
    'binary_straight','binary_curve','binary_slider','binary_shoot',
    'binary_fork','binary_changeup','binary_sinker','binary_cutball'
], axis=1)

In [10]:
lgb_param = {
        'objective' : 'binary',
        'boosting_type': 'gbdt',
        'metric' : 'binary_logloss',
        'seed' : 0,
        'learning_rate' : 0.1,
        'lambda_l1': 6.9923570049658075, 
        'lambda_l2': 0.002378623984798833, 
        'num_leaves': 18, 
        'feature_fraction': 0.45199999999999996, 
        'bagging_fraction': 0.9799724836460725, 
        'bagging_freq': 4, 
        'min_child_samples': 20
    }

## straight

In [11]:
lgb_train = lgb.Dataset(train_d, train['binary_straight'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_straight = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 1212
Best CV score: 0.6115575376416865


### curve

In [12]:
lgb_train = lgb.Dataset(train_d, train['binary_curve'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_curve = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 487
Best CV score: 0.20629562362633708


### slider

In [13]:
lgb_train = lgb.Dataset(train_d, train['binary_slider'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_slider = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 862
Best CV score: 0.39413008418905504


### shoot

In [14]:
lgb_train = lgb.Dataset(train_d, train['binary_shoot'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_shoot = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 687
Best CV score: 0.15487035007762692


### fork

In [15]:
lgb_train = lgb.Dataset(train_d, train['binary_fork'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_fork = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 701
Best CV score: 0.19118529894761357


### changeup

In [16]:
lgb_train = lgb.Dataset(train_d, train['binary_changeup'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_changeup = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 528
Best CV score: 0.1385044978012746


### sinker

In [17]:
lgb_train = lgb.Dataset(train_d, train['binary_sinker'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_sinker = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 124
Best CV score: 0.028127482936482536


### cutball

In [18]:
lgb_train = lgb.Dataset(train_d, train['binary_cutball'])

# CV
cv_results = lgb.cv(lgb_param, lgb_train,
                    num_boost_round=15000,
                    early_stopping_rounds=100,
                    #verbose_eval=100,
                    nfold=4)

num_boost_round = len(cv_results['binary_logloss-mean'])
print('Best num_boost_round:', num_boost_round)
best_cv_score = cv_results['binary_logloss-mean'][-1]
print('Best CV score:', best_cv_score)

# 学習
best_iter = int(num_boost_round * 1.1)
lgb_model = lgb.train(lgb_param, lgb_train, num_boost_round=best_iter)

# 予測
pr_cutball = lgb_model.predict(all_d, num_iteration = lgb_model.best_iteration)

Best num_boost_round: 601
Best CV score: 0.12615541852944698


## 結果出力

In [19]:
submodel = pd.DataFrame({
    'predict_straight': pr_straight,
    'predict_curve': pr_curve,
    'predict_slider': pr_slider,
    'predict_shoot': pr_shoot,
    'predict_fork': pr_fork,
    'predict_changeup': pr_changeup,
    'predict_sinker': pr_sinker,
    'predict_cutball': pr_cutball
})
print(submodel.shape)
submodel.head()

(778767, 8)


,predict_straight,predict_curve,predict_slider,predict_shoot,predict_fork,predict_changeup,predict_sinker,predict_cutball
0,0.826574,0.020212,0.059413,0.249197,0.000165,0.003005,0.000011,0.000031
1,0.636906,0.082154,0.117277,0.114695,0.005939,0.030791,0.000015,0.000080
2,0.472835,0.179000,0.083079,0.039162,0.039172,0.043636,0.000014,0.000041
3,0.613967,0.134526,0.168836,0.138130,0.000099,0.000220,0.000012,0.000041
4,0.491045,0.153934,0.219845,0.092219,0.000436,0.000573,0.000013,0.000068


In [20]:
submodel.to_feather(OUT_SUBMODEL)

### GCS

In [21]:
'gsutil cp ./{} gs://yasunobu_baseball/intermediate/'.format(OUT_SUBMODEL)

'gsutil cp ./intermediate/binary_predict_19.f gs://yasunobu_baseball/intermediate/'